In [31]:
import pymysql
wikiconn  = pymysql.connect(host='localhost',
                            user='u15068',                 # get from replica.my.cnf
                            password='G5V2xrWbdFTSkyoU',
                            port=4711,
                            db='enwiki_p',
                            charset='utf8')
wikiCur = wikiconn.cursor()

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [28]:
import datetime
import pandas as pd

def PrintableTimestamp():
    now = datetime.datetime.now().time()
    return "[{:02d}:{:02d}:{:02d}]".format(now.hour,now.minute,now.second)

def BatchQueryFromList(listOfIDs,columnThatContainsIDs,tableToQuery,listOfColumnsToRetrieve):
    '''listOfIDs is a list of keys to retrieve from an SQL table
    columnThatContainsIDs is the column in that table that stores the keys
    tableToQuery is the table containing the information you want
    listOfColumnsToRetrieve is a list containing the SQL DB table column names you want to retrieve
    Returns a pandas dataframe with column names equivalent to those in the DB you queried'''
    baseQ = "SELECT {} FROM {} WHERE {} IN ({})"
    colsForQ = ",".join(listOfColumnsToRetrieve)
    idChunks = chunks(listOfIDs, 1000)
    result_df = pd.DataFrame()
    for ix, chunk in enumerate(idChunks):
        print(PrintableTimestamp() + " fetching chunk {}, first element {}, last element {}".format(ix,chunk[0],chunk[-1]))
        idsInChunk = ",".join(map(str,chunk))
        preppedQ = baseQ.format(colsForQ,tableToQuery,columnThatContainsIDs,idsInChunk)
        wikiCur.execute(preppedQ)
        if not result_df.empty:
            result_df = result_df.append(pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve))
        else:
            result_df = pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve)
    return result_df


In [7]:
# get IDS for EPAs from github file. For NEPAs: ../Sampling/071816_batch1_sampled_ids.csv
ids2Q = pd.read_csv("../WikiSQL/070916_edit_protected_articles.csv")  # For EPAs: ../WikiSQL/070916_edit_protected_articles.csv
ids = list(map(str, ids2Q["page_id"].values)) 
print(len(ids))

7526


In [32]:
range(1,12001)
df = BatchQueryFromList(ids,'page_id','page',['page_id','page_title'])

[10:17:06] fetching chunk 0, first element 1, last element 1000
[10:17:06] fetching chunk 1, first element 1001, last element 2000
[10:17:06] fetching chunk 2, first element 2001, last element 3000
[10:17:06] fetching chunk 3, first element 3001, last element 4000
[10:17:07] fetching chunk 4, first element 4001, last element 5000
[10:17:07] fetching chunk 5, first element 5001, last element 6000
[10:17:07] fetching chunk 6, first element 6001, last element 7000
[10:17:07] fetching chunk 7, first element 7001, last element 8000
[10:17:07] fetching chunk 8, first element 8001, last element 9000
[10:17:07] fetching chunk 9, first element 9001, last element 10000
[10:17:07] fetching chunk 10, first element 10001, last element 11000
[10:17:08] fetching chunk 11, first element 11001, last element 12000


In [33]:
df

,page_id,page_title
0,10,b'AccessibleComputing'
1,12,b'Anarchism'
2,13,b'AfghanistanHistory'
3,14,b'AfghanistanGeography'
4,15,b'AfghanistanPeople'
5,18,b'AfghanistanCommunications'
6,19,b'AfghanistanTransportations'
7,20,b'AfghanistanMilitary'
8,21,b'AfghanistanTransnationalIssues'
9,23,b'AssistiveTechnology'
